In [1]:
from matplotlib import pyplot as plt
import numpy as np
import scipy as sp
import os
import glob
import sys
from pathlib import Path
from scipy.optimize import minimize, Bounds

In [2]:
# allow for import of my python modules
module_path = str(Path.cwd().parents[0] / "python")

if module_path not in sys.path:
    sys.path.append(module_path)

from modules import datahandling, spatial

In [3]:
years = ['2015', '2016', '2017']
vid_data = {}
radars = {}

for y in years:
    begin = f'{y}-08-01'
    end = f'{y}-11-15'
    path = f'/home/fiona/radar_data/vpi/night_only/{y}0801T0000_to_{y}1130T2359'
    vid_data[y], radars[y], t_range = datahandling.load_data(path, 'vid', begin, end, '1H', mask_days=True)

In [7]:
y = '2016'
names = radars[y]
space = spatial.Spatial(names)
adj, voronoi, G = space.voronoi(plot=False)

# graph without sink nodes
G = space.subgraph('type', 'measured')

data = np.stack([v.data.flatten() for k, v in vid_data[y].items()])

In [8]:
# create train-test pairs
check = np.isfinite(data).all(axis=0)
tidx = np.append(np.logical_and(check[:-1], check[1:]), False)
X = data[:,tidx]
Y = np.roll(data, -1, axis=1)[:,tidx]

scale = X.max(axis=0) * 0.2
X /= scale
Y /= scale

test_size = 200
X_train = X[:, test_size:].T
Y_train = Y[:, test_size:]
X_test  = X[:, :test_size].T
Y_test  = Y[:, :test_size]

In [9]:
targets = [n for n, data in G.nodes(data=True) if data.get('boundary')]
sources = {n : list(G.neighbors(n)) for n in targets}
sources

{0: [1, 5, 9],
 1: [0, 5, 8],
 3: [6, 13, 19],
 7: [2, 11, 14, 15, 20],
 8: [1, 5, 10, 19],
 9: [0, 4, 5, 11, 14],
 13: [3, 6, 17],
 14: [7, 9, 11],
 15: [7, 20, 21],
 17: [6, 12, 13, 18],
 18: [2, 12, 17, 20, 21],
 19: [3, 6, 8, 10],
 21: [15, 18, 20]}

In [25]:
res, targets, sources = run_optimization(X_train, Y_train, G, constraints=True)
for i, node in enumerate(targets):
    out_flux = res.x[np.where(sources==node)].sum()
    print(f'{i}: total flux = {out_flux}, \n weights = {res.x[np.where(sources==node)]}')

NameError: name 'der' is not defined

In [24]:
def run_optimization(X_train, Y_train, G, init='static', constraints=True, ftol=0.001, maxiter=200):
    
    # add self loops to graph
    G.add_edges_from([(n,n) for n in G.nodes])
        
    targets = [n for n, data in G.nodes(data=True) if data.get('boundary')]
    sources = np.concatenate([list(G.neighbors(n)) for n in targets])

    x0 = np.random.rand(sources.size)

    cons = {'type': 'eq',
            'fun' : lambda x: np.array([1-x[np.where(sources==i)[0]].sum() for i in targets])}

    method = 'SLSQP'
    bounds = Bounds(0, 1)

    #method = 'L-BFGS-B'
    #method = 'TNC'
    #method = 'trust-constr'
    
    if constraints:
        res = minimize(loss, x0, args=(X_train, Y_train, G, targets), 
                       bounds=bounds, method=method, constraints=cons, 
                       jac=jac, options={'ftol': ftol, 'disp': True, 'maxiter':maxiter})
    else:
        res = minimize(loss, x0, args=(X_train, Y_train, G, targets), bounds=bounds, method=method, 
                       jac=jac,  options={'ftol': ftol, 'disp': True, 'maxiter':maxiter})
    
    return res, targets, sources


In [23]:
def loss(w, X_train, Y_train, G, targets):
    
    idx = 0
    loss = 0
    for node in targets:
        neighbours = list(G.neighbors(node))
        X = X_train[:,neighbours]
        y = Y_train[node]
        nn = len(neighbours)
        loss += np.square(y - w[idx:idx+nn].T.dot(X.T)).sum() 
        idx += nn
    return loss
    
def jac(w, X_train, Y_train, G, targets):
    jac = []
    idx = 0
    for node in targets:
        neighbours = list(G.neighbors(node))
        X = X_train[:,neighbours]
        y = Y_train[node]
        nn = len(neighbours)
        s = y - w[idx:idx+nn].T.dot(X.T)
        jac.append(-2 * s.T.dot(X))
        idx += nn
    jac = np.concatenate(tuple(jac), axis=0)
    return der